In [2]:
%%capture
!pip install langchain_community chromadb langchain_groq

In [ ]:

from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://vijayasriiyer.medium.com/a-quick-introduction-to-llm-alignment-cad91d09c032",),

)
docs = loader.load()


from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50)

splits = text_splitter.split_documents(docs)


from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)


In [4]:
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=hf_embeddings)


retriever = vectorstore.as_retriever()

In [5]:
from langchain.prompts import ChatPromptTemplate


template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (5 queries):"""

prompt_q = ChatPromptTemplate.from_template(template)

In [ ]:
import os
os.environ['GROQ_API_KEY']=""

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_groq import ChatGroq

In [8]:
llm=ChatGroq(temperature=0)

In [9]:
generate_queries_chain=(
    prompt_q
    |llm
    |StrOutputParser()
    |(lambda x:x.split("\n"))
)

In [10]:
query="what is llm allignment?"
generate_queries_chain.invoke(query)

['1. "What is the meaning of LLM alignment in machine learning?"',
 '2. "How does LLM alignment work in natural language processing?"',
 '3. "What is the significance of LLM alignment in language models?"',
 '4. "What are the different types of LLM alignment techniques?"',
 '5. "What are the applications of LLM alignment in artificial intelligence?"']

In [11]:
from langchain.load import loads,dumps
def rrf(document:list[list],k=60):
  scores={}
  for sub_doc in document:
    for rank ,doc in enumerate(sub_doc):
      doc_str=dumps(doc)
      if doc_str not in scores:
        scores[doc_str]=0
      else:
        scores[doc_str]+=1/(k+rank)
  sorted_docs=sorted(scores.items(),key=lambda item:item[1],reverse=True)
  sorted_docs=[loads(doc) for doc,_ in sorted_docs]
  return sorted_docs


In [12]:
retrieve_chain=generate_queries_chain|retriever.map()|rrf

In [13]:
rag_template="""Generate answer to the user input query based on the context.
here is the context:{context}
here is the query:{query}

"""
prompt=ChatPromptTemplate.from_template(rag_template)

rag_chain=(
    {"context":retrieve_chain,"query":RunnablePassthrough()}
    |prompt
    |llm
    |StrOutputParser()
)


In [14]:
query="what is llm allignment?"
rag_chain.invoke(query)

<ipython-input-11-3093935de6b0>:12: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  sorted_docs=[loads(doc) for doc,_ in sorted_docs]


"LLM alignment is a process that trains a large language model (LLM) to ensure that the generated outputs align with human values and goals. This is also called preference optimization, which involves adjusting the LLM's behavior to better match human preferences. There are various alignment methods in research literature, but three popular ones include RLHF (Reinforcement Learning with Human Feedback), which involves training an LLM in two steps using human feedback."

In [15]:
dense_retriever = retriever
dense_retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x788b8f740ad0>, search_kwargs={})

In [16]:
!pip -q install rank_bm25

In [17]:
##Reranking uing Ensemble retriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
bm2retriever=BM25Retriever.from_documents(splits)
ensemble_retriever=EnsembleRetriever(
    retrievers=[bm2retriever,dense_retriever],
    weights=[0.5,0.5]
)



In [18]:

query="what is llm allignment?"
ensemble_retriever.invoke(query)

[Document(metadata={'source': 'https://vijayasriiyer.medium.com/a-quick-introduction-to-llm-alignment-cad91d09c032', 'title': 'A Quick Introduction to LLM Alignment | by Vijayasri Iyer | Medium', 'description': 'With the rise of LLMs, we have also seen a term called alignment becoming highly popularized in AI literature. Although the field of study existed before, the rise of alignment techniques such as…', 'language': 'en'}, page_content='helpful, safe, and reliable as possible. “ In the context of LLMs specifically, alignment is a process that trains an LLM to ensure that the generated outputs align with human values and goals. This is also called as alignment with respect to human preferences, hence “preference optimization”.What'),
 Document(metadata={'source': 'https://vijayasriiyer.medium.com/a-quick-introduction-to-llm-alignment-cad91d09c032', 'title': 'A Quick Introduction to LLM Alignment | by Vijayasri Iyer | Medium', 'description': 'With the rise of LLMs, we have also seen a

In [19]:
retrieve__rerank_data=generate_queries_chain|ensemble_retriever.map()|rrf

In [20]:
data=retrieve__rerank_data.invoke(query)

In [21]:
rag_chain=(
    {"context":retrieve__rerank_data,"query":RunnablePassthrough()}
    |prompt
    |llm
    |StrOutputParser()
)

In [22]:

query="what is llm allignment?"
rag_chain.invoke(query)

"LLM alignment is the process of training a large language model (LLM) to ensure that the generated outputs align with human values and goals. This is also called preference optimization, which means optimizing the LLM's preferences to match human preferences. There are various methods for LLM alignment, and one of the popular ones is Reinforcement Learning with Human Feedback (RLHF). RLHF involves training an LLM in three steps: pre-training for the base model, supervised/instruction fine-tuning for a chat variant, and using an ancillary language model to learn human preferences through a preference dataset. The ancillary language model can be much smaller than the main LLM. The LLM is then trained to distinguish between good and bad responses based on the human preferences learned."

In [23]:
##Long Context Reorder
from langchain_community.document_transformers import  LongContextReorder
reordering=LongContextReorder()



In [24]:
data=retrieve__rerank_data.invoke(query)
data

[Document(metadata={'source': 'https://vijayasriiyer.medium.com/a-quick-introduction-to-llm-alignment-cad91d09c032', 'title': 'A Quick Introduction to LLM Alignment | by Vijayasri Iyer | Medium', 'description': 'With the rise of LLMs, we have also seen a term called alignment becoming highly popularized in AI literature. Although the field of study existed before, the rise of alignment techniques such as…', 'language': 'en'}, page_content='preferences, hence “preference optimization”.What are the current methods for LLM alignment?You will find many alignment methods in research literature, we will only stick to 3 alignment methods for the sake of discussion.RLHF (Reinforcement Learning with Human Feedback):Step 1 & 2: Train an LLM'),
 Document(metadata={'source': 'https://vijayasriiyer.medium.com/a-quick-introduction-to-llm-alignment-cad91d09c032', 'title': 'A Quick Introduction to LLM Alignment | by Vijayasri Iyer | Medium', 'description': 'With the rise of LLMs, we have also seen a t

In [25]:
retrieved_docs=reordering.transform_documents(data)

In [26]:
retrieved_docs

[Document(metadata={'source': 'https://vijayasriiyer.medium.com/a-quick-introduction-to-llm-alignment-cad91d09c032', 'title': 'A Quick Introduction to LLM Alignment | by Vijayasri Iyer | Medium', 'description': 'With the rise of LLMs, we have also seen a term called alignment becoming highly popularized in AI literature. Although the field of study existed before, the rise of alignment techniques such as…', 'language': 'en'}, page_content='helpful, safe, and reliable as possible. “ In the context of LLMs specifically, alignment is a process that trains an LLM to ensure that the generated outputs align with human values and goals. This is also called as alignment with respect to human preferences, hence “preference optimization”.What'),
 Document(metadata={'description': 'With the rise of LLMs, we have also seen a term called alignment becoming highly popularized in AI literature. Although the field of study existed before, the rise of alignment techniques such as…', 'language': 'en', '

In [27]:
# You're using the pipe operator (|) to create a chain, and the dictionary you're
# passing to it is being interpreted as a RunnableParallel which expects its steps
#  to be Runnable, Callable, or dict. Since retrieved_docs is a lis
rag_chain=(
    {"context":RunnablePassthrough(),"query":RunnablePassthrough()}
    |prompt
    |llm
    |StrOutputParser()
)

query="what is llm allignment?"
rag_chain.invoke({"context":retrieved_docs,"query":query})

"LLM alignment is the process of training a large language model (LLM) to ensure that its generated outputs align with human values and goals. This is also referred to as alignment with respect to human preferences, or preference optimization. There are various methods for LLM alignment, with one popular method being Reinforcement Learning with Human Feedback (RLHF). RLHF involves training an LLM in steps 1 and 2, followed by using an ancillary language model to learn human preferences in step 3. This can be done using a preference dataset, where the LLM is the agent and the reward model provides a positive or negative reward based on how well the LLM's responses align with human preferred responses. However, implementation of LLM alignment is not straightforward and requires a lot of human expertise and compute."

In [28]:
##self querying (metadata retriever)
from langchain_core.documents import Document

docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "thriller",
            "rating": 9.9,
        },
    ),
]
vectorstore = Chroma.from_documents(docs, hf_embeddings)

In [29]:
from langchain.chains.query_constructor.base import AttributeInfo

In [30]:
metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie. One of ['science fiction', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated']",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]

In [66]:
%%capture
!pip install lark

In [31]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
document_content_description = "Brief summary of a movie"
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
)

In [32]:
retriever.invoke("I want to watch a movie rated higher than 8.5")

[Document(metadata={'director': 'Satoshi Kon', 'rating': 8.6, 'year': 2006}, page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea'),
 Document(metadata={'director': 'Andrei Tarkovsky', 'genre': 'thriller', 'rating': 9.9, 'year': 1979}, page_content='Three men walk into the Zone, three men walk out of the Zone')]

In [1]:
# !pip install lark

In [33]:
retriever.invoke("Has Greta Gerwig directed any movies about women")

[Document(metadata={'director': 'Greta Gerwig', 'rating': 8.3, 'year': 2019}, page_content='A bunch of normal-sized women are supremely wholesome and some men pine after them')]

In [35]:
rag_chain=(
    {"context":retriever,"query":RunnablePassthrough()}
    |prompt
    |llm
    |StrOutputParser()
)
query="I want to watch a movie rated higher than 8.5"


rag_chain.invoke(query)

'Based on the provided context, I would recommend the movie "Stalker" directed by Andrei Tarkovsky. It has a rating of 9.9, which is higher than 8.5.'

In [36]:
query="Has Greta Gerwig directed any movies about women"
rag_chain.invoke(query)

'Yes, Greta Gerwig has directed a movie about women. The movie is "Little Women" which was released in 2019 and has a rating of 8.3. The movie features a group of normal-sized women who are portrayed as supremely wholesome, and some men pine after them.'